In [ ]:
from IPython.display import clear_output
from demo import *

In [ ]:
%%time
# get a list of available tkrs
a = getTkrs()
adf = pd.DataFrame(a)
adf

In [ ]:
%%time
countTS()

In [ ]:
%%time
countPts()

In [ ]:
a[100] # print the 100th tkr(retieved in previous step)

In [ ]:
# Get the close series and print first and last values
tkr=a[40]

ts = ds.get_time_series('{}.close'.format(tkr))
cal = ts.get_calendar()
print(ts.get_name())
print("{} {:2.2f}".format(ts.get_first_date(), ts.get_double(ts.get_first_date_int())))
print("{} {:2.2f}".format(cal.to_date( ts.get_first_date_int()+1), ts.get_double(ts.get_first_date_int()+1)))
print('...')
print("{} {:2.2f}".format(cal.to_date(ts.get_last_date_int()-1), ts.get_double(ts.get_last_date_int()-1)))
print("{} {:2.2f}".format(ts.get_last_date(), ts.get_double(ts.get_last_date_int())))

In [ ]:
tkr=a[50]
#tkr='DE'

# Get ALL history, but display only 2 weeks
b = getOHLCV(tkr)
print("Tkr: {}".format(tkr.upper()))
b

In [ ]:
%%time
# Just time retrieval and instantiation of TS
tkr='spy'
ts = ds.get_time_series("{}.close".format(tkr))

In [ ]:
# CREATE Graph and Table output for selected Tkr in Dropdown @ selected Frequency

# setup dropdowns for tkr and freq
textbox = widgets.Dropdown(options=getTkrs(), value='A', description='Symbol:')
tbox2 = widgets.Dropdown(options=[('Daily',native_calendar_type.business()),
                                  ('Weekly',native_calendar_type.weekly_friday()),
                                  ('Monthly',native_calendar_type.monthly()),
                                  ('Quarterly',native_calendar_type.quarterly_december()),
                                  ('Annually',native_calendar_type.annual_december())], index=3,description='Frequency:')
hbox1 = widgets.HBox([textbox,tbox2])

# create bar chart widget
g = go.FigureWidget(data=[go.Line(name='Close')], 
                    layout=go.Layout(autosize=False, width=800,title=dict(text='Closing Prices ({})'.format(tkr.upper())),
                                    margin=dict(b=0, t=50,l=50,r=50,pad=2)))
g2 = go.FigureWidget(data=[go.Bar(name='Volume')], 
                     layout=go.Layout(autosize=False, width=800, height=250,title=dict(text='Volumes ({})'.format(tkr.upper())),
                                    margin=dict(b=50, t=50,l=50,r=50,pad=2)))

# capture stdout|err to display
out = widgets.Output()
display(out)

# callback for tkr/freq selected 
@out.capture()
def response(change):
    tkr = textbox.value
    ts = ds.get_time_series("{}.close".format(tkr))
    tsv = ds.get_time_series("{}.volume".format(tkr))
    tsa = time_scale_adapter_factory.get_adapter( ts, tbox2.value )
    tsva = time_scale_adapter_factory.get_adapter( tsv, tbox2.value )
    b = tsa.begin(); e = tsa.end(); bv = tsva.begin()
    index = []; values = []; vols = []
    while b != e:
        index.append( b.get_date() )
        values.append( [ "{}".format(b.get_date()), b.get_double()] )
        vols.append( [ "{}".format(bv.get_date()), bv.get_double()] )
        b.next(); bv.next()
    df = pd.DataFrame(values, index=index, columns=['Date','Close'])
    dfv = pd.DataFrame(vols, index=index, columns=['Date','Volume'])
    with g.batch_update():
        g.data[0].x = df['Date']
        g.data[0].y = df['Close']
        g2.data[0].x = dfv['Date']
        g2.data[0].y = dfv['Volume']
        g.layout.title.text='Closing Prices ({})'.format(tkr.upper())
        g2.layout.title.text='Volumes ({})'.format(tkr.upper())
    clear_output( wait=True )
    display(hbox1)
    #print('\n')
    g.show()
    g2.show()
    table = ff.create_table(getOHLCV(tkr))
    table.layout.autosize=False
    table.layout.width=800
    #print("Previous 2 weeks:\n")
    table.show()

# register callbacks for tkr/freq selection
textbox.observe(response, names='value')
tbox2.observe(response, names='value')
textbox.index=3
# show tkr/freq dropdowns
display(hbox1)

In [ ]:
help(go.Bar)